In [168]:
import numpy as np
import scipy
import scipy.fftpack as fft

import sys
sys.path.append('../')
from common import *

import torch
import torch_dct as dct
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.optim.lr_scheduler import StepLR

import torchvision
import torchvision.transforms as transforms
from torchvision import models

In [169]:
def unfold(A):
    return torch.tensor(bcirc(A)[:, :A.shape[1]])

def fold(A, shape):
    return torch.tensor(np.reshape(A, shape))

def bcirc(A):
    s = A.shape
    bcirc_A = np.zeros((s[0] * s[2], s[1] * s[2]))
    A_matriced = np.reshape(np.transpose(A, [0, 1, 2]), (s[0] * s[2], s[1]))
    for k in range(s[2]):
        bcirc_A[:, k * s[1] : (k + 1) * s[1]] = np.roll(A_matriced, k * s[0], axis=0)
    return torch.tensor(bcirc_A)

def t_product(A, B):
    dct_A = dct.dct(A)
    dct_B = dct.dct(B)
    dct_C = torch.zeros(A.shape[0], B.shape[1], A.shape[2])
    for k in range(A.shape[2]):
        dct_C[..., k] = dct_A[..., k] @ dct_B[..., k]
    return torch.tensor(dct.idct(dct_C))

def t_product_v2(A, B):
    shape = [A.shape[0], B.shape[1], A.shape[2]]
    return torch.tensor(fold(bcirc(A) @ unfold(B), shape))

def objective_function(outputs, C):
    return torch.norm(C - outputs) ** 2 / 2

In [276]:
A = torch.rand(2, 3, 4)
B = torch.rand(3, 5, 4)
C = torch.rand(2, 5)
D = torch.rand(5, 5, 4)

In [277]:
print(fft.dct(np.array(A)) - np.array(dct.dct(A)))

[[[ 0.0000000e+00  1.4901161e-08  0.0000000e+00 -1.4901161e-08]
  [ 0.0000000e+00 -5.9604645e-08  0.0000000e+00  1.1920929e-07]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 -1.1920929e-07]]

 [[ 0.0000000e+00  1.4901161e-08  0.0000000e+00 -1.4901161e-08]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  3.7252903e-09]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]]]


In [278]:
E = t_product(A, B)
t_product(E, D)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


tensor([[[109.1029, 108.3608, 108.0490, 109.1149],
         [102.9536, 102.0753, 102.8634, 103.5111],
         [118.8796, 120.4292, 120.6319, 119.1123],
         [ 91.4567,  90.2142,  91.0926,  91.4754],
         [111.0466, 110.0189, 109.2627, 110.6241]],

        [[113.0708, 111.2088, 111.2240, 112.9554],
         [106.4739, 105.4831, 105.1709, 106.9399],
         [122.5860, 123.6918, 123.2501, 122.8943],
         [ 94.0933,  92.8866,  92.6624,  94.3410],
         [114.3657, 112.2729, 112.4617, 114.0144]]])

In [259]:
class Transform_Layer(nn.Module):
    def __init__(self, size_in, size_out, n):
        super().__init__()
        self.size_in = size_in
        self.size_out = size_out
        weights = torch.rand(size_out, size_in, n)
        bias = torch.rand(size_out, 1, n)
        self.weights = nn.Parameter(weights)
        self.bias = nn.Parameter(bias)
        
    def forward(self, x):
        Wx = t_product(self.weights, x).to(device)
        return torch.add(Wx, self.bias)

In [269]:
class Transform_Net(nn.Module):
    def __init__(self):
        super(Transform_Net, self).__init__()
        self.features = nn.Sequential(
            Transform_Layer(28, 16, 128),
            nn.ReLU(inplace=True),
            #Transform_Layer(16, 12, 128),
            #nn.ReLU(inplace=True),
            # Transform_Layer(16, 10, 128),
        )

    def forward(self, x):
        x = self.features(x)
        x = dct.idct(x)
        x = torch.sum(F.log_softmax(x, dim=1), axis=1)
        
        return x

In [270]:
trainloader, testloader = load_mnist()

model = Transform_Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9, weight_decay=5e-4)

def train_step_transform(epoch, train_acc, model, trainloader, optimizer):
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    train_loss = 0
    correct = 0
    total = 0
    train_acc = []
    criterion = nn.CrossEntropyLoss()
    
    print('\nEpoch: ', epoch)
    print('|', end='')
    for batch_idx, (inputs, labels) in enumerate(trainloader):   
        inputs = inputs.squeeze()
        inputs = torch.transpose(inputs, 0, 2)
        inputs = torch.transpose(inputs, 0, 1).to(device)
        labels = labels.to(device)
        #targets = torch.zeros(10, 128)
        #for j in range(targets.shape[1]):
        #    targets[i[1][j], j] = 1
        #inputs, targets = inputs.to(device), targets.to(device)
        if not inputs.shape[2] == 128:
            continue

        
        optimizer.zero_grad()
        outputs = model(inputs)
        print(outputs)
        loss = criterion(torch.transpose(outputs, 0, 1), labels)
        loss.backward()
        
        optimizer.step()
        train_loss += loss.item()
        _, predicted = outputs.max(0)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        if batch_idx % 10 == 0:
            print('=', end='')
    print('|', 'Accuracy:', 100. * correct / total,'% ', correct, '/', total)
    train_acc.append(correct / total)
    
def train_transform(i, model, trainloader, testloader, optimizer):
    train_acc = []
    test_acc = []
    scheduler = StepLR(optimizer, step_size=5, gamma=0.9)
    for epoch in range(i):
        s = time.time()
        train_acc = train_step_transform(epoch, train_acc, model, trainloader, optimizer)
        # test_acc, _ = test(test_acc, model, testloader)
        scheduler.step()
        e = time.time()
        print('This epoch took', e - s, 'seconds to train')
        print('Current learning rate: ', scheduler.get_last_lr()[0])
    print('Best training accuracy overall: ', max(test_acc))
    return train_acc, test_acc

==> Loading data..


In [271]:
train_transform(20, model, trainloader, testloader, optimizer)


Epoch:  0
|tensor([[-27697.8926,  -3948.7805,  -5625.8989,  ...,   -103.8578,
           -103.3520,   -102.8667],
        [-28093.1504,  -4006.5271,  -5568.8608,  ...,   -111.7997,
            -99.6266,   -100.8963],
        [-28500.7812,  -4072.5767,  -5660.5713,  ...,   -115.3003,
           -105.2653,    -99.3140],
        ...,
        [-27886.3008,  -3974.2185,  -5595.7441,  ...,    -99.6970,
           -146.1663,   -112.4078],
        [-28089.1660,  -4028.7615,  -5601.0986,  ...,   -105.4042,
           -102.5223,   -110.2952],
        [-27794.9141,  -3981.5056,  -5551.2129,  ...,   -121.8633,
           -105.3973,   -100.8923]], device='cuda:0', grad_fn=<SumBackward1>)
=tensor([[-26871.4434,  -3733.9688,  -5307.6079,  ...,   -108.6834,
           -100.5483,    -98.9826],
        [-27236.3008,  -3819.2751,  -5499.8330,  ...,   -116.6707,
           -109.6091,   -107.9723],
        [-27668.0859,  -3857.9419,  -5567.2607,  ...,   -106.9280,
           -109.8920,   -178.0916],
     

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


tensor([[-26550.5117,  -3666.1963,  -5248.6182,  ...,    -99.7950,
           -100.6818,    -99.4391],
        [-26531.8125,  -3753.3809,  -5326.4531,  ...,    -99.3516,
            -95.7179,    -99.0060],
        [-26947.2422,  -3786.1184,  -5383.4004,  ...,    -98.2939,
           -124.2143,   -112.9248],
        ...,
        [-26510.6914,  -3690.1653,  -5292.1123,  ...,   -115.7153,
           -105.5032,    -99.5904],
        [-26764.1582,  -3775.8735,  -5368.7690,  ...,   -113.9205,
            -96.2489,   -102.9120],
        [-26456.6113,  -3701.9856,  -5296.6865,  ...,   -104.9386,
           -115.6209,    -97.1204]], device='cuda:0', grad_fn=<SumBackward1>)
tensor([[-28461.7148,  -4039.4221,  -5602.6328,  ...,    -97.3947,
            -97.3721,   -102.3243],
        [-28531.8047,  -4092.1877,  -5728.6362,  ...,    -96.1919,
            -95.2210,   -109.7178],
        [-29006.1406,  -4143.6562,  -5892.4595,  ...,   -134.0677,
           -115.4386,   -100.0791],
        ...,
     

tensor([[-28172.2578,  -3896.9163,  -5415.2124,  ...,   -111.0823,
           -110.7522,    -96.3401],
        [-28213.2715,  -3987.5107,  -5731.2031,  ...,   -108.1947,
            -98.8523,   -106.5010],
        [-28666.3418,  -4041.7754,  -5809.8525,  ...,    -96.4946,
           -114.7759,    -99.3390],
        ...,
        [-28080.9023,  -3933.6475,  -5678.5840,  ...,   -104.2115,
           -120.0958,   -106.4868],
        [-28397.6309,  -4007.1040,  -5664.4268,  ...,   -119.8614,
           -110.6204,   -102.4622],
        [-28102.9453,  -3955.0569,  -5617.0469,  ...,   -102.9085,
           -112.3636,   -105.3164]], device='cuda:0', grad_fn=<SumBackward1>)
tensor([[-27473.4922,  -3915.5935,  -5606.1416,  ...,   -101.7995,
           -107.2471,   -118.8536],
        [-27635.7598,  -3974.5752,  -5483.4839,  ...,    -97.3449,
            -97.0126,   -120.4476],
        [-28085.3906,  -4042.5439,  -5607.6880,  ...,   -112.5196,
           -120.2363,   -115.8220],
        ...,
     

tensor([[-24663.3418,  -3750.6106,  -5027.3955,  ...,   -113.0627,
            -99.7885,   -126.6148],
        [-24936.5117,  -3822.4595,  -5000.6333,  ...,    -95.6880,
           -105.5549,   -119.1233],
        [-25314.1562,  -3861.1274,  -5025.2168,  ...,   -109.4901,
           -108.4183,   -101.5530],
        ...,
        [-24789.3242,  -3768.2065,  -4972.2148,  ...,   -100.0158,
           -118.3875,   -112.5276],
        [-25067.1367,  -3835.9561,  -5078.8613,  ...,   -101.1760,
            -94.8870,   -117.8821],
        [-24769.0098,  -3796.7683,  -4957.7632,  ...,   -100.6659,
            -98.1625,   -102.8384]], device='cuda:0', grad_fn=<SumBackward1>)
tensor([[-25070.0098,  -3536.2524,  -5112.1523,  ...,   -104.4627,
           -103.4173,    -97.6597],
        [-25196.3398,  -3599.0437,  -4972.4175,  ...,   -107.2891,
           -101.1323,   -111.4810],
        [-25481.0391,  -3629.8972,  -5082.6001,  ...,   -109.0928,
            -97.6241,   -115.1010],
        ...,
     

...

tensor([[-26678.1758,  -3712.9109,  -5339.0771,  ...,   -105.3205,
           -100.2919,   -105.9425],
        [-26753.6738,  -3754.3320,  -5399.9707,  ...,   -105.8830,
            -96.8440,    -99.1047],
        [-27101.1797,  -3814.1741,  -5469.2944,  ...,   -100.4977,
           -115.7078,   -116.9635],
        ...,
        [-26556.6797,  -3697.6738,  -5335.8516,  ...,   -107.9850,
           -106.4783,   -102.6897],
        [-26847.0938,  -3775.8354,  -5380.6440,  ...,   -100.9485,
           -100.8694,   -121.5639],
        [-26601.1055,  -3735.3262,  -5396.5171,  ...,    -98.8080,
           -107.2287,   -101.8301]], device='cuda:0', grad_fn=<SumBackward1>)
tensor([[-26153.6484,  -3792.0859,  -5277.2822,  ...,   -101.0188,
           -103.6035,   -102.7643],
        [-26374.9062,  -3887.8284,  -5280.9932,  ...,   -106.9509,
            -99.2424,   -112.3525],
        [-26690.0781,  -3924.5181,  -5374.6484,  ...,   -113.5108,
            -98.4763,   -123.7832],
        ...,
     

tensor([[-28192.3789,  -3814.9873,  -5673.8203,  ...,   -101.0092,
           -107.1350,   -118.8691],
        [-28439.1719,  -3881.6484,  -5627.1357,  ...,   -106.3858,
            -96.7402,   -148.5062],
        [-28789.4961,  -3929.4482,  -5748.8955,  ...,    -97.5563,
           -151.5889,   -110.7031],
        ...,
        [-28206.7129,  -3836.7510,  -5676.6206,  ...,   -102.5650,
           -107.3316,    -98.5539],
        [-28500.0039,  -3922.2847,  -5683.2744,  ...,   -114.1114,
           -106.5010,    -98.7955],
        [-28205.7754,  -3851.6711,  -5579.8608,  ...,   -104.3493,
            -97.0789,   -102.0686]], device='cuda:0', grad_fn=<SumBackward1>)
tensor([[-30266.5332,  -3457.6240,  -6125.5156,  ...,    -96.2458,
            -99.1428,   -110.9294],
        [-30242.9258,  -3515.4578,  -6034.2168,  ...,    -97.9363,
            -98.3818,   -104.3195],
        [-30734.0137,  -3565.8569,  -6146.7090,  ...,   -102.4725,
           -101.2706,   -110.3645],
        ...,
     

           -113.0341,   -127.7116]], device='cuda:0', grad_fn=<SumBackward1>)
tensor([[-27093.2578,  -3752.6074,  -5366.5879,  ...,    -96.0307,
           -100.9246,   -106.5418],
        [-27063.8848,  -3791.0564,  -5451.6309,  ...,   -101.9485,
            -96.3648,   -106.7849],
        [-27434.7207,  -3855.8774,  -5527.4961,  ...,    -97.5570,
           -104.8651,   -104.1083],
        ...,
        [-27091.0820,  -3735.2437,  -5399.3091,  ...,   -102.9447,
           -101.6246,   -100.0771],
        [-27331.4062,  -3798.8843,  -5441.4951,  ...,   -100.6368,
           -100.4535,   -102.1235],
        [-26932.1328,  -3786.7031,  -5397.9971,  ...,   -136.3498,
           -108.9560,    -97.2329]], device='cuda:0', grad_fn=<SumBackward1>)
tensor([[-29307.4375,  -4019.0371,  -5853.5874,  ...,    -98.5897,
           -100.3024,    -98.2523],
        [-29142.8164,  -4082.4729,  -5868.3242,  ...,   -108.1451,
           -115.7802,   -105.3222],
        [-29661.3750,  -4134.3550,  -5971.6

tensor([[-27435.9922,  -3576.3904,  -5542.0132,  ...,   -107.2256,
           -105.1972,   -100.6711],
        [-27547.1523,  -3624.5205,  -5518.1133,  ...,    -99.2175,
           -109.8758,   -123.2603],
        [-27981.7910,  -3690.2322,  -5660.1343,  ...,   -103.4595,
           -114.3569,   -107.6970],
        ...,
        [-27430.7793,  -3592.4375,  -5475.1816,  ...,    -99.1526,
           -118.5176,    -96.0953],
        [-27809.9590,  -3650.5442,  -5527.9995,  ...,   -105.0534,
           -104.4484,   -145.0869],
        [-27332.5703,  -3613.7080,  -5469.6357,  ...,   -109.3510,
           -100.1567,    -98.6735]], device='cuda:0', grad_fn=<SumBackward1>)
tensor([[-27020.1328,  -3920.5483,  -5480.3447,  ...,   -101.1127,
           -102.6547,   -111.5900],
        [-27067.2246,  -3987.9805,  -5437.7954,  ...,    -99.9556,
           -105.4231,   -108.9794],
        [-27401.8555,  -4046.9407,  -5502.3462,  ...,   -103.5825,
           -103.2555,   -120.3190],
        ...,
     

KeyboardInterrupt: 